### imports

In [2]:
pip install jpype1


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tabula
import pandas as pd

### code

In [4]:
# Open the PDF file
pdf_path = '/Users/AmandaLi/Desktop/Courses/STA141B/cleaned-ea-survey.pdf'

In [5]:
# Extract tables from PDF into a list of DataFrames
tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)

FileNotFoundError: [Errno 2] JVM DLL not found: /Library/Java/JavaVirtualMachines/jdk-14.0.2.jdk/Contents/Home/lib/libjli.dylib

In [166]:
# convert into pandas df
table0 = tables[0]
table1 = tables[1]
table2 = tables[2]

In [167]:
# remove first line
table0 = rm_first(table0)
table1 = rm_first(table1)
table2 = rm_first(table2)
# change column names
table0 = rename_cols(table0)
table1 = rename_cols(table1)
table2 = rename_cols(table2)

In [169]:
table2 = table2.iloc[:-1]

In [171]:
def rename_cols(table):
    table.columns = ["County", 
                         "District", 
                         "Total County Pop", 
                         "Total District Pop", 
                         "County Pop in District", 
                         "Percent of County Pop in District", 
                         "Percent of District Pop in County"]
    return table

In [172]:
def rm_first(table):
    return table.iloc[1:]

In [173]:
def reload_tables(table0, table1, table2):
    tables = [table0, table1, table2]
    return tables

In [213]:
cleaned_tables = reload_tables(table0, table1, table2)
combined_df = pd.concat(cleaned_tables, ignore_index=True)

In [214]:
combined_df['County'] = combined_df['County'].fillna(method='ffill')

/var/folders/xd/ljd80glj4631l10fr9j779bm0000gn/T/ipykernel_44250/4216097951.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df['County'] = combined_df['County'].fillna(method='ffill')


In [215]:
# changing column types for easy data analysis
combined_df['District'] = combined_df['District'].astype(float)
combined_df['Total County Pop'] = combined_df['Total County Pop'].str.replace(',', '').astype(int)
combined_df['Total District Pop'] = combined_df['Total District Pop'].str.replace(',', '').astype(int)
combined_df['County Pop in District'] = combined_df['County Pop in District'].str.replace(',', '').astype(int)
combined_df['Percent of County Pop in District'] = combined_df['Percent of County Pop in District'].str.replace('%', '').astype(float) / 100
combined_df['Percent of District Pop in County'] = combined_df['Percent of District Pop in County'].str.replace('%', '').astype(float) / 100

In [218]:
# create dictionary
combined_dict = {}

# populate dictionary
for index, row in combined_df.iterrows():
    county = row['County']
    
    # for current row
    district_info = {
        'District': row['District'],
        'Total County Pop': row['Total County Pop'],
        'Total District Pop': row['Total District Pop'],
        'County Pop in District': row['County Pop in District'],
        'Percent of County Pop in District': row['Percent of County Pop in District'],
        'Percent of District Pop in County': row['Percent of District Pop in County']
    }
    
    # if county is already a key, append info
    if county in combined_dict:
        combined_dict[county].append(district_info)
    else:
        # if not already a key, create new key
        combined_dict[county] = [district_info]